Instalacja potrzebnych bibliotek tensorflow i keras

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} tensorflow

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           2 KB  defaults
    absl-py-0.8.0              |           py37_0         157 KB  conda-forge
    astor-0.7.1                |             py_0          22 KB  conda-forge
    c-ares-1.15.0              |    h516909a_1001         100 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    gast-0.3.2                 |             py_0          11 KB  conda-forge
    google-past

In [2]:
import sys
!conda install --yes --prefix {sys.prefix} keras

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    keras-2.2.5                |           py37_1         538 KB  conda-forge
    libgpuarray-0.7.6          |    h14c3975_1003         263 KB  conda-forge
    pygpu-0.7.6                |py37h3010b51_1000         683 KB  conda-forge
    theano-1.0.4               |py37hf484d3e_1000         3.6 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  keras              conda-forge/linux-64::keras-2.2.5-py37_1
  libgpuarr

Import bibiliotek tensorflow i keras

In [4]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


Zdefiniowanie funkcji która posłuży za export wyuczonego modelu

In [5]:
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', 'human_detection_graph.pbtxt')

    saver.save(K.get_session(), 'out/human_detection.chkp')

    freeze_graph.freeze_graph('out/human_detection_graph.pbtxt', None, False, 'out/human_detection.chkp', output_node_name, "save/restore_all", "save/Const:0", 'out/frozen_human_detection.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_human_detection.pb', "rb") as f: input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(input_graph_def, input_node_names, [output_node_name], tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_human_detection.pb', "wb") as f: f.write(output_graph_def.SerializeToString())

Definicja stałych oraz ustawienie o jakiej strukturze będą pliki graficzne

In [6]:
IMG_WIDTH, IMG_HEIGHT = 100, 100

EPOCHS = 50
BATCH_SIZE = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

Stworzenie struktury sieci neuronowej

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Skonfigurowanie generatorów zestawów danych

In [8]:
train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

Wygenerowanie większej ilości danych wejściowych

In [9]:
train_generator = train_datagen.flow_from_directory('data/train', target_size = (IMG_WIDTH, IMG_HEIGHT), batch_size = BATCH_SIZE, color_mode = "rgb", class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory('data/test', target_size = (IMG_WIDTH, IMG_HEIGHT), batch_size = BATCH_SIZE, color_mode = "rgb", class_mode = 'categorical')
model.fit_generator(train_generator, steps_per_epoch=1832 // BATCH_SIZE, epochs = EPOCHS, validation_data = validation_generator, validation_steps = 741 // BATCH_SIZE)

Found 3338 images belonging to 2 classes.
Found 977 images belonging to 2 classes.

Epoch 1/50
114/114 [==============================] - 23s 203ms/step - loss: 0.6495 - acc: 0.7039 - val_loss: 0.7044 - val_acc: 0.4212
Epoch 2/50
114/114 [==============================] - 17s 153ms/step - loss: 0.6413 - acc: 0.6975 - val_loss: 0.9924 - val_acc: 0.1678
Epoch 3/50
114/114 [==============================] - 18s 160ms/step - loss: 0.7530 - acc: 0.6945 - val_loss: 1.0998 - val_acc: 0.2067
Epoch 4/50
114/114 [==============================] - 18s 158ms/step - loss: 3.4955 - acc: 0.6974 - val_loss: 13.3390 - val_acc: 0.1644
Epoch 5/50
114/114 [==============================] - 17s 153ms/step - loss: 7.7567 - acc: 0.5155 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/50
114/114 [==============================] - 17s 149ms/step - loss: 10.9262 - acc: 0.3177 - val_loss: 2.5346 - val_acc: 0.8419
Epoch 7/50
114/114 [==============================] - 16s 142ms/step - loss: 11.6449 - acc: 0.2736 - val

Zapisywanie wag

In [10]:
model.save_weights('weights.h5')

A następnie eksportowany jest model do katalogu out

In [11]:
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Sigmoid")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from out/human_detection.chkp
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
Instructions for updating:
Use tf.gfile.GFile.
